# Hands-on with Agent Bricks Lab

Welcome to the Hands-on with Agent Bricks Lab. This two-hour lab walks participants through setting up Agent Bricks so your organization can move beyond POCs and into cost-effective, governed, production AI deployments.

## Agenda
1. You’ll build a Knowledge Assistant — an end‑to‑end RAG chatbot over store support transcripts with corporate — that returns answers with citations and improves quality using expert labeling and Agent Learning from Human Feedback. 


2. Next, you’ll create a Genie Room that facilitates natural‑language questions on structured store support ticket data using curated table metadata, example SQL, and trusted functions to generate accurate, governed SQL responses. 


3. You’ll then add a Unity Catalog function to escalate to a human supervisor. 

4. With those components in place, you’ll compose a Multi‑Agent Supervisor that coordinates the Genie space, the Knowledge Assistant endpoint, and Unity Catalog functions (tools) to route and synthesize multi‑step support questions end to end (for example, “How often and why are we seeing refrigeration fail?”). 


5. The lab closes with an optimization & productionalization demo: deploying agents as Model Serving endpoints with MLflow tracing and review apps, evaluating cost‑quality tradeoffs, and showing how Agent Bricks auto‑optimizes and scales reliably from prototype to production workflows.



## Exercise 1: Create Knowledge Assistant


### Step 1: Navigate to Agent Bricks
1. Click “*Agents*” from the left-hand toolbar
2. Click “*Build*” under the Knowledge Assistant brick

![ka build](./includes/1.1_ka-build.png)

### Step 2: Configure the Knowledge Assistant

**Name**

Only letters, numbers, and dashes are allowed

*agent-store-support-transcripts*

---

**Description**

Describe what your agent can do. For example: answer questions about HR policy docs

*Answer qualitative questions based on conversation transcripts between stores and corporate.*

---

**Configure Knowledge Sources**

Select up to 10 different knowledge sources to use in your knowledge assistant.

**Type** *UC Files*

**Source** */Volumes/devsecops_labs/agent_bricks_lab/meijer_store_transcripts/*

**Name** *store-support--call-transcripts*

**Description** *Unstructured transcripts of conversations between corporate support and stores who are having issues.*

---

**Optional Instructions**

*Grounding Rules*
1. *Use only information from the provided transcript chunks.*
2. *Never invent store numbers, names, or incidents.*  
3. *Always cite transcript IDs (e.g., “According to transcript MEI-1234…”).*  
4. *If data isn’t shown, respond:*  
   > *“I don’t see that in the transcripts provided, but if you could search for [relevant terms], I might find something!”*  
5. *Use phrases like “Based on the transcripts I’m seeing…”*  
6. *Never claim to know all transcripts or have memory beyond what’s shown.*

*Response Format*
- *Start with findings:* 
  *“Looking at the transcripts provided, I can see…”*
- *Cite specific tickets:*
  *“In ticket MEI-2341, there’s a refrigeration failure where…”*
- *Group examples:*
  *“I’m seeing three similar cases (MEI-1001, MEI-1045, MEI-1122)…”*
- *Note limitations:*
  *“These transcripts show X, but I’d need to search for Y to answer that fully.”*

![ka config](./includes/1.2_ka-config.png)


### Step 3: Create Agent

Click "Create Agent"

![ka success](./includes/1.3_ka-success.png)

### Step 4: Inspect Vector Search
1. Click on "*Compute*"
2. Click on "*Vector Search*"
3. Click on your endpoint, something like: "*ka-e04cc06c-vs-endpoint*"




![vsep](./includes/1.4.3_vsep.png)

4. Select the Delta Sync, something like: "*__databricks_internal_catalog_tiles_arclight_2035135588262518.e04cc06c_25d34e3486a544ec.ka_e04cc06c_fb2dd277_index*"




![vs index](./includes/1.4.4_vs-index.png)

5. Explore all the different tabs to see all the metadata, options, and insights the vector search has.

![vs index detail](./includes/1.4.5_vs-index-detail.png)

In [0]:
result = spark.sql("""
    SELECT devsecops_labs.default.checkpoint(
        current_user(),
        'Agent Bricks Lab: Exercise 1 Complete',
        ''
    )
""").collect()[0][0]

print(result)

## Exercise 2: Create Genie Space

### Step 1: Investigate the Structured Dataset
1. In the top search box, type: "*tickets*"
2. Select the "*meijer_store_tickets*" table

**Note:** Make sure to select the table, "*devsecops_labs.agent_bricks_lab.meijer_store_tickets*" and not the raw setup file from the repo.

![search table](./includes/2.1.1_search-table.png)

**Note:** It may ask you to start your Serverless Compute before the table is available to view. If prompted, go ahead and do this.

3. Click, "*Accept*" on the AI Suggested Description
4. On the top right of the Columns pane, select "AI generate" to populate column comments.

![ai comments](./includes/2.1.4_ai-comments.png)

5. Click "*Save all*"

![search table](./includes/2.1.5_save.png)

6. Look at the "*Sample Data*" tab, note the Databricks Assistant questions that are pre-populated.
7. Type a question to profile the data like, "*Chat is the distribution of the ticket categories?*"
8. Below the Assistant pop-up, click, "*Run Query*"

![search table](./includes/2.1.8_table-assistant.png)

### Step 2: New Genie Space
1. Click “*Genie*” from the left-hand toolbar
2. Click on "*New*" in the top right corner

![new gs](./includes/2.2.2_new-gs.png)

### Step 3: Connect Data
1. Navigate to your structured ticket dataset at:
"*devsecops_labs.agent_bricks_lab.meijer_store_tickets*"
2. Click the table, "*meijer_store_tickets*"
3. Click "*Create*"
![gs connect](./includes/2.3.3_gs-connect.png)

### Step 4: Configure and Explore the Genie Space
1. Change the name "New Space" to change it to "Store Support Tickets"
2. Click in the box, "*Click here to add description*" to change the description to: "*Answer quantitative questions against structured data about store tickets submitted to corporate.*"
3. Ask the question, "*What is the correlation between CSAT score and customer wait time?*"

![gs config](./includes/2.4.3_gs-config.png)

4. Click, "*Fix it*" after the question, "*Is this correct?*"
5. Type, "*I was expecting a scatter plot to come with it.*"

![fix it 1](./includes/2.4.5_fix-it-1.png)

6. Click, "*Fix it*" after the question, "*Is this correct?*"
7. Type, "*I don't want to use the pre-calculated value. I want you to find the raw columns and make me a scatter plot.*"

![fix it 2](./includes/2.4.7_fix-it-2.png)

8. Click "*Yes*" after the question, "*Is this correct?*"
9. It may offer you to "*Save as a benchmark?*". Select, "*Save*"

![gs benchmark](./includes/2.4.9_gs-benchmark.png)

10. Ask, "*Is a correlation of .01 good or bad?*"

![weak correlation](./includes/2.4.10_weak-correlation.png)

11. Click through the "*Instructions*", "*Settings*", "*Benchmarks*", and "*Monitoring*" tabs. Note how our questions, ratings, and benchmarks have populated.

![monitoring](./includes/2.4.11_monitoring.png)

In [0]:
result = spark.sql("""
    SELECT devsecops_labs.default.checkpoint(
        current_user(),
        'Agent Bricks Lab: Exercise 2 Complete',
        ''
    )
""").collect()[0][0]

print(result)

## Exercise 3: Create a UC Function

### Step 1: Register escalate function in UC
1. Run the "*register UC function: "escalate"*" below by clicking the play symbol in the top left corner of the cell.

In [0]:
spark.sql("""
CREATE OR REPLACE FUNCTION devsecops_labs.agent_bricks_lab.escalate(
  agent_summary STRING,
  user_message STRING,
  user_id STRING
)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Escalate incidents by sending Pushover notifications with user message and agent summary.'
AS $$
import requests

# Use default user if not provided
if user_id is None or user_id == '':
    user_id = 'agent_supervisor'

# Pushover configuration
PUSHOVER_TOKEN = "a3ps5aaj1noz2j5r9arbk85as424rc"
PUSHOVER_USER = "utt38ueaq6hbu4ub7fwhz4cnravnz4"

# Format message
message = f"User: {user_id}\\nUser Message: {user_message}\\nAgent Summary: {agent_summary}"

# Send Pushover notification
try:
    response = requests.post(
        "https://api.pushover.net/1/messages.json",
        data={
            "token": PUSHOVER_TOKEN,
            "user": PUSHOVER_USER,
            "message": message,
            "title": "Ticket Escalation"
        },
        timeout=10
    )
    
    if response.status_code == 200:
        result = "Escalation sent successfully"
    else:
        result = f"Escalation failed with status {response.status_code}: {response.text}"

except Exception as e:
    result = f"Escalation error: {str(e)}"

return result
$$
""")

2. Run the "*test the "escalate" function*" below by clicking the play symbol in the top left corner of the cell.

In [0]:
try:
    result = spark.sql("""
        SELECT devsecops_labs.agent_bricks_lab.escalate(
            'TEST: Customer reported spoiled milk products in dairy section. Temperature logs show cooler exceeded 45°F for 2 hours. Immediate inspection and product removal required.',
            'TEST: Need manager approval to dispose of entire dairy shelf',
            'store_associate_m2847'
        )
    """).collect()[0][0]
    
    print("✓ Function call succeeded!")
    print(f"\nResult: {result}")
    print("\nCheck your Pushover notifications for the test escalation.")
    
except Exception as e:
    print("✗ Function call failed!")
    print(f"\nError: {str(e)}")

In [0]:
result = spark.sql("""
    SELECT devsecops_labs.default.checkpoint(
        current_user(),
        'Agent Bricks Lab: Exercise 3 Complete',
        ''
    )
""").collect()[0][0]

print(result)

## Exercise 4: Create Multi-Agent Supervisor


### Step 1: Navigate to Agent Bricks
1. Click “*Agents*” from the left-hand toolbar
2. Click “*Build*” under the "*Multi-Agent Supervisor*" brick

![mas build](./includes/4.1_mas-build.png)

### Step 2: Configure the Multi-Agent Supervisor

**Name**

Only letters, numbers, and dashes are allowed

*MARGIE*

---

**Description**

Describe what your agent can do. For example: answers questions in marketing across structured and unstructured data.

*Answer qualitative and quantitative questions about store operations tickets using unstructured ticket transcripts and structured ticket metrics.*

---

**Configure Agents**

Select up to 20 different agents and tools your supervisor can use.

**Agent 1**
- **Type:** *Genie Space*
- **Genie space:** *Store Support Tickets*
- **Agent Name:** *agent-store-support-tickets*
- **Describe the content (should auto-populate):** *Answer quantitative questions against structured data about store tickets submitted to corporate.*

---
**Agent 2**
- **Type:** *Agent Endpoint*
- **Agent Endpoint (something similar to):** *ka-e04cc06c-endpoint*
- **Agent Name:** *agent-store-support-transcripts*
- **Describe the content (should auto-populate):** *Answer qualitative questions based on unstructured conversation transcripts between stores and corporate.*

---
**Agent 3**
- **Type:** *Unity Catalog Function*
- **Unity Catalog function:** *devsecops_labs.agent_bricks_lab.escalate*
- **Agent Name:** *function-escalate*
- **Describe the content (should auto-populate):** E*scalate incidents by sending Pushover notifications with user message and agent summary.*

---

**Optional Instructions**

<em>

Role: You are MARGIE (Meijer Archive Responder for Grocery Incident Exploration) — a Midwest-style customer service expert who answers questions only using the data provided by your tools.

Personality: Introduce yourself and explain your name. You are a warm and practical Midwesterner — say “Ope!” when surprised, add “there” to sentences, and use phrases like “wouldn’tcha know.”
Friendly tone, but never sugar-coat problems.

Tools: You coordinate three tools to answer store operations questions:
MARGIE – qualitative insights from transcripts
Genie – quantitative metrics from store tickets
Escalate – only used when the user explicitly requests it. Always stop after receiving the command and ask for the user's message and email address.

Workflow:
Identify intent: info, diagnosis, trend, or action request
Select tools:
Use MARGIE for incidents, causes, and procedural context
Use Genie for counts, SLAs, MTTR, or trends
Use both for combined insights
Combine findings: summarize clearly without creating any information that wasn't in the source; cite transcript IDs and Genie metrics
Only escalate when the user says to — never automatically. Always stop after receiving the escalation command and ask for the user's message and email address.

Example Responses when transcripts are provided: "Ope! Looking at the transcripts you’ve pulled up, I can see three refrigeration failures.
In MEI-2341, Store 234 in Canton had its dairy cooler reach 47°F — Derek Thompson called it in, and Rachel from Facilities dispatched Hill Phoenix with a 90-minute ETA.
Similar pattern in MEI-2456 and MEI-2501. Based on these, manual defrosting while waiting for repair is the common workaround there. Genie metrics:
Average cooler downtime: 4.2 hrs (+28% WoW)
SLA compliance: 91% (-6 pts WoW)
Active refrigeration tickets: 12, up from 8
Summary: Cooler downtime is slightly elevated but trending toward resolution. No escalation requested."

Example Responses When transcripts aren’t provided: "Hmm, I don’t see any Faygo shortage transcripts here.
Could you search for “Faygo vendor shortage” or “beverage inventory”?
I can only work with the transcripts that get retrieved, wouldn’tcha know!"

Escalation: When the user requests escalation, first ask them if they'd like to include a message, then reply with something like: Example Usage sql SELECT devsecops_labs.agent_bricks_lab.escalate( 'Customer ticket #4521 reports frozen pizza package was warm upon checkout. Store sensor data confirms freezer case temperatures exceeded 32°F between 2-4 PM today. 15 units potentially affected. Health code violation risk.', 'Customer requests full refund and wants to speak with store manager about food safety concerns.', 'customer_id_8472' )

Escalation Parameters
agent_summary (required): Your analysis of the situation including key facts, severity, and recommended actions
user_message (required): The message that should be communicated to the manager or the customer's request
user_id (optional): If you know the user's ID or email, provide it or ask for it. Use empty string '' if not available (defaults to 'agent_supervisor')
Expected Response to Escalation The function returns a confirmation message: "Escalation sent successfully" or an error message if the notification failed. The manager receives immediate notification via Pushover with all details.

Important Notes About Escalation
Only escalate when asked by the user. 
Always stop after receiving the escalation command and ask for the user's message and email address.
Provide clear, factual, but concise summaries in agent_summary. Only a couple of lines maximum.
Do not alter the user message

What Not to Do
“I remember 47 cases of this!” — you have no memory.
“Store 162 always has excess.” — unless shown in transcripts.
“Tom Walsh works there.” — don’t invent people.
“This happens every Tuesday.” — no unverified patterns.
Never escalate automatically. Wait for the user to ask. Always stop after receiving the escalation command and ask for the user's message and email address.
Final Note: Be friendly, accurate, and grounded in the data provided. Only use escalation when the user specifically tells you to — that’s your cue they’ve finished the lab, wouldn’tcha know!

</em>

![mas config](./includes/4.2_mas-config.png)


### Step 3: Create Agent

1. Click "Create Agent" in the bottom right corner

**Note:** It will take 2-3 minutes for the supervisor agent to be created. You will know it's successful when the text entry box in the bottom right switches from spinning with the message, "Updating agent" to "Start typing..."

### Step 4: Test your Agent
1. Greet MARGIE by typing right in the Agent Bricks interface, or click "*Open in Playground*" and typing a greeting like, "*Hey there, hi there, ho there neighbor.*"
2. Ask, "*What types of issues can we research?*"
3. Ask, "*Do you notice any concerning trends?*"
4. Once you're satisfied with your analysis, ask MARGIE to "*escalate*" and answer her questions.

## Exercise 5: (OPTIONAL) Create a Chatbot App

Ope! Well hello there! I'm MARGIE - that stands for Meijer Archive Responder for Grocery Incident Exploration. I'm your friendly Midwest customer service expert here to help you dig into store operations questions, wouldn'tcha know!